In [131]:
import torch 
import torch.nn.functional as F 
from torch import nn 
from torchsummary import summary

In [174]:
class basicNet(nn.Module): 
    def __init__(self, num_kernels=40):
        super().__init__() 
        
        self.cnn = nn.Sequential(
            # input 229x626
            nn.Conv2d(1, 10, (3, 3), padding=1),
            nn.BatchNorm2d(10), 
            nn.ReLU(), 
            # current shape 229x10x626
            nn.Conv2d(10, 10, (3, 3), padding=1), 
            nn.BatchNorm2d(10), 
            nn.ReLU(), 
            nn.MaxPool2d((4, 1)), 
            nn.Dropout(0.25), 
            # current shape 57x626x10
            nn.Conv2d(10, 20, (3, 3), padding=1), 
            nn.BatchNorm2d(20), 
            nn.ReLU(), 
            nn.MaxPool2d((4, 1)), 
            nn.Dropout(0.25), 
            # current shape 14x626x20
        )
    
        self.fc1 = nn.Sequential(
            nn.Linear((626*20*14), 3130),
            nn.Dropout(0.5),
            # Code calls for another batch norm here but unsure what to specifically normalize here
            nn.Sigmoid(),
        )
        
        self.rnn = nn.GRU(input_size=626, hidden_size=626)
        
        self.fc2 = nn.Sequential(
            nn.Linear((626*5), 626*3), 
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        batch_size = x.size(0)
        print("hello!")
        print(x.shape)
        x = self.cnn(x)
        print(x.shape)
        x = x.transpose(1, 3).flatten(-3)
        print(x.shape)
        x = self.fc1(x)
        print(x.shape)
        x = x.reshape(batch_size, 5, 626) 
        print(x.shape)
        a, x = self.rnn(x)
        print(x.shape)
        print(a.shape)
        x = x.transpose(1, 2).flatten(-2)
        print(x.shape)
        print(a.shape)
        x = self.fc2(x)
        print(x.shape)
        return x

In [175]:
inp = torch.randn(1, 1, 229, 626)
inp2 = torch.randn(1, 626, 14*40)
model = basicNet()
res = model(inp)

hello!
torch.Size([1, 1, 229, 626])
torch.Size([1, 20, 14, 626])
torch.Size([1, 175280])
torch.Size([1, 3130])
torch.Size([1, 5, 626])
torch.Size([1, 5, 626])
torch.Size([1, 5, 626])
torch.Size([1, 3130])
torch.Size([1, 5, 626])
torch.Size([1, 1878])


In [176]:
summary(model, (1, 229, 626))

hello!
torch.Size([2, 1, 229, 626])
torch.Size([2, 20, 14, 626])
torch.Size([2, 175280])
torch.Size([2, 3130])
torch.Size([2, 5, 626])
torch.Size([1, 5, 626])
torch.Size([2, 5, 626])
torch.Size([1, 3130])
torch.Size([2, 5, 626])
torch.Size([1, 1878])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 10, 229, 626]             100
       BatchNorm2d-2         [-1, 10, 229, 626]              20
              ReLU-3         [-1, 10, 229, 626]               0
            Conv2d-4         [-1, 10, 229, 626]             910
       BatchNorm2d-5         [-1, 10, 229, 626]              20
              ReLU-6         [-1, 10, 229, 626]               0
         MaxPool2d-7          [-1, 10, 57, 626]               0
           Dropout-8          [-1, 10, 57, 626]               0
            Conv2d-9          [-1, 20, 57, 626]           1,820
      BatchNorm2d-10          [-1, 20, 57, 6

In [61]:
x = torch.randn(1, 1, 229, 626)
print(x.shape)
print(torch.transpose(x, 2, 3).shape)

torch.Size([1, 1, 229, 626])
torch.Size([1, 1, 626, 229])


In [62]:
inp = torch.randn(1, 1, 229, 626)
print(inp)
print(inp.shape)
inpMod = inp.transpose(2, 3)
print(inpMod) 
print(inpMod.shape)


tensor([[[[ 0.4273,  0.4385, -0.4127,  ..., -0.3558,  0.9284, -1.0684],
          [-0.9351, -0.6404, -1.0334,  ...,  1.0659,  0.5425, -0.8897],
          [ 0.5311, -1.0317, -0.9953,  ...,  1.0791,  0.7940, -0.8141],
          ...,
          [-0.0255, -0.4855,  1.0145,  ...,  0.0353,  0.1047, -0.2250],
          [ 1.0807,  1.8722,  0.5847,  ..., -1.1226,  0.6154,  0.4500],
          [-0.4431, -0.0150,  1.6142,  ..., -0.7368, -1.1152, -0.0807]]]])
torch.Size([1, 1, 229, 626])
tensor([[[[ 0.4273, -0.9351,  0.5311,  ..., -0.0255,  1.0807, -0.4431],
          [ 0.4385, -0.6404, -1.0317,  ..., -0.4855,  1.8722, -0.0150],
          [-0.4127, -1.0334, -0.9953,  ...,  1.0145,  0.5847,  1.6142],
          ...,
          [-0.3558,  1.0659,  1.0791,  ...,  0.0353, -1.1226, -0.7368],
          [ 0.9284,  0.5425,  0.7940,  ...,  0.1047,  0.6154, -1.1152],
          [-1.0684, -0.8897, -0.8141,  ..., -0.2250,  0.4500, -0.0807]]]])
torch.Size([1, 1, 626, 229])


In [147]:
x1 = torch.randn(1, 3130)
x2 = x1.reshape(626, 5)
print(x2)
print(x2.shape)

tensor([[-0.1345,  0.0615, -0.6616, -0.1899, -1.1562],
        [-2.2821, -1.1816,  1.9076, -0.0721, -0.5117],
        [-0.1722, -1.5611, -1.5708,  1.9354,  1.4404],
        ...,
        [ 1.6196,  0.7744, -0.1988,  0.2942,  0.4754],
        [ 0.9557, -0.1949, -0.0298,  0.5104,  0.8935],
        [ 0.2450,  1.1709, -1.4639, -1.0854, -0.9646]])
torch.Size([626, 5])
